In [1]:
import pickle
import numpy as np

data_dir = '/home/sangwon/Projects/ai_coach/train_ma_dnn/data'
file_name = 'sc2_2s3z_100.pkl'

with open(f'{data_dir}/{file_name}', 'rb') as f:
  expert_trajs = pickle.load(f)

wons = expert_trajs["wons"]
lengths = expert_trajs["lengths"]

print(np.mean(wons), np.std(wons))
print(np.mean(lengths), np.std(lengths))
rewards = expert_trajs["rewards"]
np_rewards = np.array(rewards)
np_sum_rewards = np.sum(np_rewards, axis=2).squeeze()
print(np_sum_rewards.shape)
print(np.mean(np_sum_rewards[:, 0]), np.std(np_sum_rewards[:, 0]))





0.97 0.1705872210923198
120.0 0.0
(100, 5)
19.930613167623523 0.9707643574108895


In [3]:
n_episodes = len(expert_trajs["latents"])
n_agents = len(expert_trajs["latents"][0])
print(n_episodes, n_agents)

n_latents = 4

latent_stats = np.zeros((n_agents, n_episodes, n_latents))

for i_e, episode in enumerate(expert_trajs["latents"]):
  for i_a, latents in enumerate(episode):
    for latent in latents:
      latent_stats[i_a, i_e, latent] += 1

print("average latent per episode:")
mean_latent_states = np.mean(latent_stats, axis=1)
print(mean_latent_states/mean_latent_states.sum(axis=-1, keepdims=True))
print(np.std(latent_stats, axis=1))



100 5
average latent per episode:
[[0.621      0.14325    0.18241667 0.05333333]
 [0.63808333 0.07283333 0.23908333 0.05      ]
 [0.63033333 0.13516667 0.16333333 0.07116667]
 [0.561      0.21616667 0.14925    0.07358333]
 [0.53216667 0.27958333 0.09591667 0.09233333]]
[[16.644807   16.22879848 10.63851023  8.1215762 ]
 [14.05507382  8.40073806 11.57730107  5.81205643]
 [12.11983498  9.68460634 11.01998185 10.54553934]
 [13.64835521 13.48986286 12.36535078  8.20616232]
 [16.90030769 19.38523923  6.48458942  9.39327419]]


In [5]:
# # convert to non-numpy version
# def convert_numpy_to_list(obj):
#     if isinstance(obj, np.ndarray):
#         return obj.tolist()  # Convert numpy array to list
#     elif isinstance(obj, list):
#         # Recursively convert elements of the list
#         return [convert_numpy_to_list(item) for item in obj]
#     elif isinstance(obj, dict):
#         # Recursively convert elements of the dictionary
#         return {key: convert_numpy_to_list(value) for key, value in obj.items()}
#     else:
#         return obj  # If it's not a numpy array or list, return as is


# expert_trajs_converted = convert_numpy_to_list(expert_trajs)

# with open(f'{data_dir}/converted_{file_name}', 'wb') as f:
#   pickle.dump(expert_trajs_converted, f)




In [28]:
np_expert_trajs = {}

for key, all_episode in expert_trajs.items():
    if isinstance(all_episode[0], list) and len(all_episode[0])==n_agents:
        np_expert_trajs[key] = np_expert_trajs.get(key, [])
        for episode in all_episode:
            list_agent_data = []
            for data in episode:
                list_agent_data.append(np.array(data))
            np_expert_trajs[key].append(list_agent_data)
    else:
        np_expert_trajs[key] = np.array(all_episode)


In [30]:
print(np_expert_trajs.keys())
key = "latents"
print(len(np_expert_trajs[key]), type(np_expert_trajs[key]))
print(len(np_expert_trajs[key][0]), type(np_expert_trajs[key][0]))
print(np_expert_trajs[key][0][0].shape, type(np_expert_trajs[key][0][0]))

key = "lengths"
print(len(np_expert_trajs[key]), type(np_expert_trajs[key]))
print(np_expert_trajs[key].shape)



dict_keys(['states', 'avail_actions', 'next_states', 'actions', 'rewards', 'dones', 'latents', 'wons', 'lengths'])
100 <class 'list'>
5 <class 'list'>
(120, 1) <class 'numpy.ndarray'>
100 <class 'numpy.ndarray'>
(100,)


In [31]:

with open(f'{data_dir}/converted_{file_name}', 'wb') as f:
  pickle.dump(np_expert_trajs, f)
